In [3]:
! pip install -q tensorflow-io
! pip install tensorflow-addons
! pip install git+https://github.com/qubvel/classification_models.git

     |████████████████████████████████| 22.7 MB 1.7 MB/s 
     |████████████████████████████████| 2.3 MB 44.4 MB/s 
     |████████████████████████████████| 679 kB 4.3 MB/s 
  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-me4nk487
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-me4nk487
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 2.6 MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20045 sha256=9c18a9bbe98d62d898be779e44557ad11c6ab5229fd94594091ba56035024b0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-12l13y83/wheels/0b/96/56/27b17c903efc647c51e4f364bfc20aa67f8d3dccad63c4fb4e
Successfully built image-classifiers


In [ ]:
tf.__version__

'2.5.0'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import librosa
import pandas as pd
import numpy as np
import pickle
!pip install tqdm
from tqdm import tqdm
import os
from tensorflow.keras.layers import Input, LSTM,GlobalAveragePooling2D, Dense,GlobalAveragePooling1D,MaxPooling2D,Dropout,BatchNormalization,Conv2D,Conv1D,MaxPooling1D,Activation,Concatenate,Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa
import os
from sklearn.model_selection import train_test_split
from classification_models.keras import Classifiers
from sklearn.metrics import label_ranking_average_precision_score
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow


In [4]:
data=pd.read_csv('/content/drive/MyDrive/train_tp.csv')
data

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20


In [5]:
# Splitting the train true positive data into train and validation
train, validation = train_test_split(data, test_size = 0.1, stratify = data['species_id'])

In [6]:
def preprocess_dbscale_mel_spectrogram(file):    
    audio = tfio.IOTensor.graph(tf.int16).from_audio(file)
    audio_slice = audio[0:]

    # remove last dimension
    audio_tensor = tf.squeeze(audio_slice, axis=[-1])
    audio_tensor = tf.cast(audio_tensor, tf.float32)

    # Convert to spectrogram
    spectrogram = tfio.audio.spectrogram(audio_tensor, nfft=2048, window=2048, stride=512)

    # Convert to mel-spectrogram
    mel_spectrogram = tfio.audio.melscale(spectrogram, rate=48000, mels=384, fmin=40, fmax=24000)

    # Convert to db scale mel-spectrogram
    mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)

    # Expanding the dimensions of spectrograms by 1 
    image = tf.expand_dims(mel_spectrogram, axis= -1)
    # Resizing the spectrogram
    image = tf.image.resize(image, [384, 768])
    # Converting the spectrogram to rgb
    image = tf.image.grayscale_to_rgb(image)
    return image

In [7]:
def preprocess_augment(spectrogram,label):
    """In this function we resize the image and perform data augumentations for train data"""

    # Getting a random value and based on the random value computing the augmentation
    a = np.random.uniform() 
    if a<0.15:
        # Flipping the spectrogram left to right
        spectrogram = tf.image.flip_left_right(spectrogram)
    elif a<0.25:
        # Flipping the spectrogram up to down
        spectrogram = tf.image.flip_up_down(spectrogram)
    elif a<0.5:
        # Adding a random contrast to spectrogram
        spectrogram = tf.image.random_contrast(spectrogram, 0.2,0.5)
    else:
        # Adding a random brightness to spectrogram
        spectrogram = tf.image.random_brightness(spectrogram, 0.2)

    return spectrogram, label


In [8]:
def preprocess_train(file):
    #print(file)
    #print(file['recording_id']+'.flac')
    image = preprocess_dbscale_mel_spectrogram('/content/drive/MyDrive/'+file['recording_id']+'.flac')
    # Computing the one hot encoded values of species_id
    label = tf.one_hot(file['species_id'], 24)
    # Returning the spectrogram and one hot encoded species_id
    return image, label


def preprocess_val(file):
   
    image = preprocess_dbscale_mel_spectrogram('/content/drive/MyDrive/'+file['recording_id']+'.flac')

    # Computing the one hot encoded values of species_id
    label = tf.one_hot(file['species_id'], 24)
    # Returning the spectrogram and one hot encoded species_id
    return image, label


In [9]:

# Creating a dataset from train data and mapping the preprocess train function 
files_ds = tf.data.Dataset.from_tensor_slices(dict(train))
train_dataset = files_ds.map(preprocess_train).cache().map(preprocess_augment).shuffle(3).batch(8).prefetch(-1).repeat()

# Creating a dataset from train data and mapping the preprocess train function 
files_ds = tf.data.Dataset.from_tensor_slices(dict(validation))
val_dataset = files_ds.map(preprocess_val).cache().shuffle(3).batch(8).prefetch(-1)

In [6]:
def lrap(y_true, y_pred):
    return tf.py_function(label_ranking_average_precision_score, (y_true, y_pred), tf.double)

In [ ]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.RepeatDataset

In [11]:
backbone = tf.keras.applications.DenseNet121(include_top = False,input_shape = (384,768,3), weights="imagenet")

for layer in backbone.layers[:0]:
    layer.trainable = False

model = tf.keras.Sequential([
            backbone,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.he_normal()),
            tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.he_normal()),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(24,bias_initializer=tf.keras.initializers.Constant(-2.))])

model.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 12, 24, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________

In [12]:

model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=2e-3,total_steps=30*140,warmup_proportion=0.3,min_lr=1e-6), loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits = True), metrics=[lrap])

In [15]:
model.fit(train_dataset,steps_per_epoch=140, validation_data=val_dataset,epochs = 60,callbacks=callback_list)


Epoch 1/60
140/140 [==============================] - 2435s 17s/step - loss: 0.6445 - lrap: 0.1566 - val_loss: 0.4716 - val_lrap: 0.1817

Epoch 00001: val_lrap improved from -inf to 0.18172, saving model to /content/drive/MyDrive/model_save_densenet/weights-01-0.1817.hdf5
Epoch 2/60
140/140 [==============================] - 84s 602ms/step - loss: 0.5203 - lrap: 0.2098 - val_loss: 0.8092 - val_lrap: 0.1582

Epoch 00002: val_lrap did not improve from 0.18172
Epoch 3/60
140/140 [==============================] - 86s 615ms/step - loss: 0.4761 - lrap: 0.2396 - val_loss: 3.3460 - val_lrap: 0.1859

Epoch 00003: val_lrap improved from 0.18172 to 0.18588, saving model to /content/drive/MyDrive/model_save_densenet/weights-03-0.1859.hdf5
Epoch 4/60
140/140 [==============================] - 87s 619ms/step - loss: 0.4593 - lrap: 0.2465 - val_loss: 0.9291 - val_lrap: 0.1770

Epoch 00004: val_lrap did not improve from 0.18588
Epoch 5/60
140/140 [==============================] - 87s 620ms/step - lo

In [16]:
model.fit(train_dataset,steps_per_epoch=140, validation_data=val_dataset,epochs = 60,callbacks=callback_list)


Epoch 1/60
140/140 [==============================] - 91s 646ms/step - loss: 0.2553 - lrap: 0.6547 - val_loss: 0.2635 - val_lrap: 0.6164

Epoch 00001: val_lrap did not improve from 0.62735
Epoch 2/60
140/140 [==============================] - 86s 614ms/step - loss: 0.2544 - lrap: 0.6527 - val_loss: 0.2621 - val_lrap: 0.6170

Epoch 00002: val_lrap did not improve from 0.62735
Epoch 3/60
140/140 [==============================] - 86s 617ms/step - loss: 0.2565 - lrap: 0.6468 - val_loss: 0.2626 - val_lrap: 0.6118

Epoch 00003: val_lrap did not improve from 0.62735
Epoch 4/60
140/140 [==============================] - 86s 616ms/step - loss: 0.2576 - lrap: 0.6400 - val_loss: 0.2629 - val_lrap: 0.6274

Epoch 00004: val_lrap improved from 0.62735 to 0.62739, saving model to /content/drive/MyDrive/model_save_densenet/weights-04-0.6274.hdf5
Epoch 5/60
140/140 [==============================] - 86s 617ms/step - loss: 0.2585 - lrap: 0.6395 - val_loss: 0.2625 - val_lrap: 0.6087

Epoch 00005: val_lr

In [10]:
!pip install --quiet efficientnet
import efficientnet.tfkeras as efn

In [14]:
import datetime
import os
tf.keras.backend.clear_session()
%load_ext tensorboard
log_dir=os.path.join("/content/drive/MyDrive/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)

from tensorflow.keras.callbacks import ModelCheckpoint
filepath="/content/drive/MyDrive/model_save_densenet/weights-{epoch:02d}-{val_lrap:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_lrap',  verbose=1, save_best_only=True, mode='max')

callback_list = [checkpoint,tensorboard_callback]



In [ ]:
inputs =Input(shape=(384,768,3))
base_model = efn.EfficientNetB3(input_tensor=inputs,include_top=False,weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x =Dropout(.5)(x)
x=tf.keras.layers.Dense(60, activation='relu', kernel_initializer=tf.keras.initializers.he_normal())(x)
x =Dropout(.6)(x)
output =Dense(24,bias_initializer=tf.keras.initializers.Constant(-2.))(x)

model = Model(inputs=inputs, outputs=output)

model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=2e-3,total_steps=30*140,warmup_proportion=0.3,min_lr=1e-6), loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits = True), metrics=[lrap])

44113920/44107200 [==============================] - 0s 0us/step


In [ ]:
model.fit(train_dataset,steps_per_epoch=140, validation_data=val_dataset,epochs =35,callbacks=callback_list)

Epoch 1/35
 13/140 [=>............................] - ETA: 1:02:43 - loss: 0.4956 - lrap: 0.1496

In [ ]:
%tensorboard --logdir logs

In [8]:
import tensorflow as tf
from tensorflow_addons.optimizers import RectifiedAdam
tf.keras.optimizers.RectifiedAdam = RectifiedAdam


model_densenet = tf.keras.models.load_model('/content/drive/MyDrive/model_save_densenet/weights-41-0.6449.hdf5', custom_objects = {'RectifiedAdam' : RectifiedAdam,'score':lrap},compile=False)

In [11]:
import tensorflow as tf
from tensorflow_addons.optimizers import RectifiedAdam
tf.keras.optimizers.RectifiedAdam = RectifiedAdam


model_efficientnet = tf.keras.models.load_model('/content/drive/MyDrive/model_save/weights-29-0.7197.hdf5', custom_objects = {'RectifiedAdam' : RectifiedAdam,'score':lrap},compile=False)

In [12]:
model_densenet.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=2e-3,total_steps=30*140,warmup_proportion=0.3,min_lr=1e-6), loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits = True), metrics=[lrap])
model_efficientnet.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=2e-3,total_steps=30*140,warmup_proportion=0.3,min_lr=1e-6), loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits = True), metrics=[lrap])

In [ ]:
for i in val_dataset:
  #print(i[0][0].shape)
  val_data=i[0]
  val_tar=i[1]
  print(val_data.shape)
  print(val_tar.shape)
  break

(8, 384, 768, 3)
(8, 24)


In [ ]:
truth=val_tar.numpy()
truth

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
scores = model.predict(val_data)
scores

array([[-3.2465618 , -3.3943036 , -4.3925257 , -2.290316  , -1.7539396 ,
        -3.5315552 , -4.4996414 , -3.8752434 , -0.484401  , -3.2973223 ,
        -2.753775  , -4.504846  , -3.8070035 , -2.3921313 , -1.9515926 ,
        -3.7625332 , -4.0075183 , -4.2396836 , -2.2933877 , -4.115899  ,
        -4.1744514 , -0.50506294, -2.3186665 , -3.210285  ],
       [-2.742435  , -3.34042   , -2.86093   , -1.7298416 , -2.9020803 ,
        -1.8732426 , -0.95581657, -2.0135436 , -3.0257254 , -1.6178907 ,
        -1.7288537 , -1.2508895 , -3.0720544 , -3.9870288 , -2.9663024 ,
        -2.1763926 , -3.3161416 , -1.57058   , -3.0576782 , -1.3970466 ,
        -1.7404428 , -3.25169   , -1.6373502 , -1.7492464 ],
       [-3.4290519 , -4.4029293 , -3.3085802 , -1.7742989 , -3.3606858 ,
        -2.096531  , -0.2673533 , -2.4322078 , -3.5526164 , -1.5294093 ,
        -1.9050455 , -1.1807325 , -3.46665   , -4.979808  , -3.5122013 ,
        -1.9108572 , -3.7977736 , -1.5024478 , -3.5247307 , -1.1565347 ,
  

In [ ]:
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits



In [ ]:
def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class

In [ ]:
#https://www.kaggle.com/pkmahan/understanding-lwlrap
score, weight = calculate_per_class_lwlrap(truth, scores)
print("Each class score", score)
print("Weight of each class", weight)
LwLRAP = (score*weight).sum()
print("LwLRAP", LwLRAP)

Each class score [0.    0.    0.    0.125 0.    1.    1.    0.    0.    0.    0.    0.
 0.    0.    0.    1.    0.    0.    0.    1.    0.    0.5   0.    1.   ]
Weight of each class [0.    0.    0.    0.125 0.    0.125 0.125 0.    0.    0.    0.    0.
 0.    0.    0.    0.125 0.    0.    0.    0.125 0.    0.125 0.    0.25 ]
LwLRAP 0.828125


In [ ]:
predicted = model.predict(val_data.numpy())
print(predicted)
list_per=[i.tolist().index(np.max(i)) for i in predicted]



[[-1.8795855  -1.009711   -2.0507715  -1.5180122  -1.4193183  -3.3795724
  -3.9589534  -2.8786724  -1.982974   -2.4006782  -3.4823356  -3.7908354
  -2.2965028  -1.1996514  -0.9100561  -2.5173197  -2.2900631  -3.7691789
  -1.5678018  -3.8859668  -3.3389845  -1.4357482  -2.7254927  -3.0755315 ]
 [-2.9020448  -1.7371556  -2.695209   -1.6467334  -3.7665436  -2.5067384
  -2.5388937  -1.5765082  -3.9240503  -2.030638   -2.6647189  -2.2745652
  -3.3041015  -3.1240602  -2.4682295  -2.8397055   0.61313987 -1.8521237
  -3.6162276  -3.0191774  -2.4631832  -3.364745   -2.8097148  -2.598744  ]
 [-4.576619   -5.372407   -4.0699077  -2.70264    -4.406909   -2.163323
  -3.6652675  -4.159107   -3.7323215  -2.5138373  -2.7592382  -3.2446318
  -3.2720413  -6.333077   -5.15286    -4.5315127  -5.4712048  -3.385694
  -4.8542557  -4.2132144  -3.7901258  -5.9710517  -2.363399    3.0014954 ]
 [-1.471442   -1.6796668  -1.7189535  -1.3481913  -2.0203722  -1.5097107
  -2.248396   -2.1254756  -2.041546   -2.074460

In [13]:
def preprocess_dbscale_mel_spectrogram(file):    
    audio = tfio.IOTensor.graph(tf.int16).from_audio('/content/drive/MyDrive/test/'+ file)
    audio_slice = audio[0:]

    # remove last dimension
    audio_tensor = tf.squeeze(audio_slice, axis=[-1])
    audio_tensor = tf.cast(audio_tensor, tf.float32)

    # Convert to spectrogram
    spectrogram = tfio.audio.spectrogram(audio_tensor, nfft=2048, window=2048, stride=512)

    # Convert to mel-spectrogram
    mel_spectrogram = tfio.audio.melscale(spectrogram, rate=48000, mels=384, fmin=40, fmax=24000)

    # Convert to db scale mel-spectrogram
    mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)

    # Expanding the dimensions of spectrograms by 1 
    image = tf.expand_dims(mel_spectrogram, axis= -1)
    # Resizing the spectrogram
    image = tf.image.resize(image, [384, 768])
    # Converting the spectrogram to rgb
    image = tf.image.grayscale_to_rgb(image)
    return image

In [14]:
test_data=os.listdir('/content/drive/MyDrive/test')
len(test_data)

1992

In [ ]:
test_data[0]

'7d064b789.flac'

In [15]:
test_img=[]
for i in tqdm(test_data):
  img = preprocess_dbscale_mel_spectrogram(i)
  test_img.append(np.array(img))


100%|██████████| 1992/1992 [1:19:51<00:00,  2.41s/it]


In [ ]:
test_img[1991].shape

(384, 768, 3)

In [20]:
predicted_test_data=[]
for i in tqdm(test_img):
  pred_model_densenet = model_densenet.predict(np.expand_dims(i, axis = 0))
  pred_model_efficientnet = model_efficientnet.predict(np.expand_dims(i, axis = 0))
  pred = (pred_model_densenet + pred_model_efficientnet) / 2
  predicted_test_data.append(pred)

100%|██████████| 1992/1992 [48:53<00:00,  1.47s/it]


In [17]:
predicted_test_data

[array([[-0.18134558, -2.0360067 , -1.7865393 , -1.6968014 , -2.39083   ,
         -1.9014617 , -3.3416235 , -2.8074446 , -1.7969773 , -2.2332344 ,
         -3.2413802 , -3.3307774 , -1.3197768 , -2.5462008 , -2.4632893 ,
         -2.7271402 , -3.0958185 , -2.9717054 , -1.2624395 , -2.6299715 ,
         -2.8140783 , -2.669487  , -2.4927018 , -2.9151163 ]],
       dtype=float32)]

In [ ]:
from numpy import savez_compressed

savez_compressed('/content/drive/MyDrive/predicted_test_data.npz', predicted_test_data)

savez_compressed('/content/drive/MyDrive/test_data.npz', test_data)

In [21]:
predicted_test_data_new=np.squeeze(predicted_test_data, axis=(1,))

In [ ]:
(predicted_test_data_new[0])

array([-0.77221954, -1.9855454 , -2.5401676 , -1.3586137 , -1.716683  ,
       -2.3526776 , -3.630341  , -3.816938  , -1.6501678 , -2.974076  ,
       -3.7760692 , -4.2330694 , -0.8994869 , -2.1062174 , -2.7587116 ,
       -3.3841586 , -2.9429111 , -3.2152195 , -1.0073056 , -2.9127116 ,
       -3.688004  , -2.613102  , -2.648941  , -2.4840145 ], dtype=float32)

In [22]:
column=['s0','s1'	,'s2'	,'s3',	's4',	's5'	,'s6','s7',	's8',	's9',	's10',	's11',	's12',	's13',	's14',	's15'	,'s16',	's17'	,'s18',	's19'	,'s20',	's21',	's22',	's23']

In [23]:
df = pd.DataFrame(data=predicted_test_data_new,index = test_data,columns=column)

In [24]:
df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
7d064b789.flac,-0.181346,-2.036007,-1.786539,-1.696801,-2.390830,-1.901462,-3.341624,-2.807445,-1.796977,-2.233234,-3.241380,-3.330777,-1.319777,-2.546201,-2.463289,-2.727140,-3.095819,-2.971705,-1.262439,-2.629972,-2.814078,-2.669487,-2.492702,-2.915116
7d4c75228.flac,-1.117904,-2.276394,-1.974131,-1.508851,-1.588506,-1.755278,-2.750446,-2.583918,-1.839616,-2.080136,-3.014987,-2.974391,-0.631959,-2.881556,-2.192395,-2.485193,-3.001122,-3.001638,-0.968302,-2.394103,-2.717012,-2.469282,-2.166744,-2.016168
7dafb75e0.flac,-1.543853,-2.609704,-1.661935,-1.421255,-2.053091,-1.055067,-2.215418,-2.254807,-2.152347,-2.290104,-2.308115,-2.512888,-1.270482,-2.770142,-2.356285,-2.346637,-2.956211,-2.839258,-1.710155,-1.922762,-2.507324,-2.901019,-1.847933,-1.606109
7d188d367.flac,-1.563805,-2.266073,-1.798391,-1.462778,-1.574582,-1.875902,-2.465223,-2.356843,-2.036975,-1.927291,-3.010244,-2.876614,-0.605860,-2.785285,-2.011417,-2.091562,-2.753160,-2.688542,-1.047076,-2.187098,-2.494048,-2.489937,-2.086056,-2.138624
7e14bfa8e.flac,-4.015090,-3.255013,-4.451508,-2.345621,-3.632944,-2.757291,-2.691586,-1.537498,-2.951644,-4.125506,0.894112,-2.683210,-5.175931,-3.140952,-2.838866,-1.400273,-3.051259,-2.999346,-4.081250,-2.494700,-3.400139,-2.881124,-2.344611,-2.070969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7c3f80ebe.flac,0.304051,-2.114228,-1.730165,-1.664178,-3.041821,-1.836403,-3.693714,-2.736755,-1.809325,-2.320501,-3.220625,-3.538252,-1.530428,-2.872643,-2.750135,-2.906721,-3.407407,-3.071846,-1.515290,-2.896374,-2.966729,-3.177056,-2.480542,-3.298629
7c424b3c9.flac,-3.307044,-2.434880,-3.138282,-1.985147,-2.200586,-2.426800,-3.496834,-3.026733,-2.657478,-2.644892,-3.048644,-3.554491,-2.495568,0.117452,-2.248529,-2.396033,-2.943964,-3.118511,-2.758161,-2.497287,-3.084730,-2.653075,-2.376633,-2.625860
7c3b73585.flac,-2.074891,-2.386307,-1.268996,-1.338863,-2.231519,-1.679492,-2.117013,-1.485750,-2.614358,-2.127113,-2.456430,-2.165810,-1.742610,-2.851687,-2.373562,-1.693890,-2.669604,-2.288677,-2.107257,-2.148349,-1.945663,-3.271209,-1.865221,-1.684505
7cfaed5a0.flac,-1.172880,-2.158128,-1.968047,-1.516912,-1.426354,-1.952752,-2.648178,-2.496339,-1.781976,-2.019371,-2.965409,-2.875727,-0.717689,-2.698814,-2.154322,-2.259155,-2.967951,-2.873453,-0.901298,-2.375186,-2.619760,-2.300500,-2.125462,-2.178924


In [25]:
df.to_csv('/content/drive/MyDrive/kaggle_test_rainforest_2.csv')